# Stage 1 Comparison: Single-Channel vs Multi-Channel Dataset
## 1D-CNN Baseline — Justification for Multi-Channel Data Collection

Trains the same **CNN_Classifier** on two 600-sample datasets under **equal sample conditions**:
- **Single-Channel**: channel 5 only — 100 samples per scenario
- **Multi-Channel**: 4 channels with distinct center frequencies (c1, c3, c4, c7) — **25 samples per channel per scenario**

Channels selected to maximise diversity across both center frequency and bandwidth:

| Channel | Center Freq | Bandwidth | Character |
|---|---|---|---|
| c1 | 3494.4 MHz | 499.2 MHz | Low-freq, narrowband |
| c3 | 4492.8 MHz | 499.2 MHz | Mid-freq, narrowband |
| c4 | 3993.6 MHz | 1331.2 MHz | Mid-freq, **wideband** |
| c7 | 6489.6 MHz | 1081.6 MHz | High-freq, **wideband** |

All four channels have **distinct center frequencies**, spanning 3.5–6.5 GHz with bandwidths ranging from 499 MHz to 1331 MHz.

Three levels of evaluation:
1. Single 70/15/15 split
2. Stratified 5-Fold CV (mean ± std)

In [ ]:
CONFIG = {
    "pre_crop": 10, "post_crop": 50, "total_len": 60,
    "search_start": 740, "search_end": 890,
    "embedding_size": 128, "input_channels": 1, "dropout": 0.4,
    "batch_size": 64, "max_epochs": 50, "lr": 1e-3,
    "weight_decay": 1e-4, "warmup_epochs": 3, "patience": 40,
    "grad_clip": 1.0, "val_ratio": 0.15, "test_ratio": 0.15, "seed": 42,
}
DATA_DIR = "../dataset/channels/"

: 

In [ ]:
import copy, math, contextlib, io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    classification_report, roc_curve, auc
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    torch.cuda.empty_cache()
print(f"Device: {device}")
torch.manual_seed(CONFIG["seed"])
np.random.seed(CONFIG["seed"])

---
## Section 1: Data Loading & Preprocessing

In [ ]:
def get_roi_alignment(sig,
                      search_start=CONFIG['search_start'],
                      search_end=CONFIG['search_end']):
    region = sig[search_start:search_end]
    if len(region) == 0:
        return np.argmax(sig)
    peak_idx = search_start + np.argmax(region)
    peak_val = sig[peak_idx]
    noise_section = sig[:search_start]
    if len(noise_section) > 10:
        threshold = max(np.mean(noise_section) + 3*np.std(noise_section), 0.05*peak_val)
    else:
        threshold = 0.05 * peak_val
    leading_edge = peak_idx
    for i in range(peak_idx, max(search_start - 20, 0), -1):
        if sig[i] < threshold:
            leading_edge = i + 1
            break
    return leading_edge


def _process_rows(df):
    """CIR preprocessing — outputs (N, 1, 60) channels-first for Conv1d."""
    PRE = CONFIG['pre_crop']; TOTAL = CONFIG['total_len']
    cir_cols = sorted([c for c in df.columns if c.startswith('CIR')],
                      key=lambda x: int(x.replace('CIR', '')))
    seqs, labels = [], []
    for _, row in df.iterrows():
        sig = pd.to_numeric(row[cir_cols], errors='coerce').fillna(0).astype(float).values
        rxpacc = float(row.get('RXPACC', 128.0))
        if rxpacc > 0:
            sig = sig / rxpacc
        le = get_roi_alignment(sig)
        start = max(0, le - PRE); end = start + TOTAL
        if end > len(sig): end = len(sig); start = max(0, end - TOTAL)
        crop = sig[start:end]
        if len(crop) < TOTAL:
            crop = np.pad(crop, (0, TOTAL - len(crop)), mode='constant')
        lo, hi = crop.min(), crop.max()
        crop = (crop - lo) / (hi - lo) if hi > lo else np.zeros(TOTAL)
        seqs.append(crop); labels.append(float(row['Label']))
    # CNN channels-first: (N, 1, 60)
    X = np.array(seqs).reshape(-1, 1, TOTAL).astype(np.float32)
    y = np.array(labels).astype(np.float32)
    return X, y


def load_cir_dataset(filepath):
    print(f'Loading: {filepath}')
    df = pd.read_csv(filepath)
    print(f'  Samples: {len(df)}')
    X, y = _process_rows(df)
    print(f'  Output: X={X.shape}, y={y.shape} | LOS={int((y==0).sum())}, NLOS={int((y==1).sum())}')
    return X, y

print("Data loading functions ready (CIR only — channels-first for CNN).")

---
## Section 2: CNN_Classifier Architecture
1D-CNN with 3 conv blocks + Global Average Pooling — no recurrence, no ODE dynamics.

In [ ]:
class CNN_Classifier(nn.Module):
    """1D-CNN baseline with GAP. No recurrence, no FP_AMPL conditioning."""
    def __init__(self, input_channels=1, embedding_size=128, dropout=0.4):
        super().__init__()
        self.embedding_size = embedding_size
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=5, padding=2),
            nn.BatchNorm1d(16), nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=5, padding=2, stride=2),
            nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, embedding_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(embedding_size), nn.ReLU(),
        )
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(embedding_size, 32), nn.SiLU(),
            nn.Dropout(dropout), nn.Linear(32, 1), nn.Sigmoid()
        )

    def forward(self, x):
        # x: (batch, 1, 60)
        features = self.encoder(x)
        embedding = self.gap(features).squeeze(-1)
        return self.classifier(embedding)

_m = CNN_Classifier()
print(f'CNN_Classifier | params: {sum(p.numel() for p in _m.parameters()):,} | embed_dim=128')
print(f'  Conv blocks: 1->16->32->128, kernel 5/5/3, stride 1/2/2')
print(f'  Pooling: Global Average Pooling')
del _m

---
## Section 3: Training Function
Early stopping (patience=40), best-model restoration.

In [ ]:
def train_model(X_train, y_train, X_val, y_val,
                config=CONFIG, verbose=True, seed=None):
    _seed = seed if seed is not None else config['seed']
    torch.manual_seed(_seed)
    np.random.seed(_seed)

    X_tr = torch.tensor(X_train).to(device)
    y_tr = torch.tensor(y_train).unsqueeze(1).to(device)
    X_va = torch.tensor(X_val).to(device)
    y_va = torch.tensor(y_val).unsqueeze(1).to(device)

    loader = DataLoader(TensorDataset(X_tr, y_tr),
                        batch_size=config['batch_size'], shuffle=True)

    model = CNN_Classifier(
        input_channels=config['input_channels'],
        embedding_size=config['embedding_size'],
        dropout=config['dropout']
    ).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'],
                            weight_decay=config['weight_decay'])
    T = config['max_epochs']; W = config['warmup_epochs']
    def lr_lambda(e):
        if e < W: return (e+1)/W
        return max(0.01, 0.5*(1+math.cos(math.pi*(e-W)/max(1,T-W))))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    history = {'train_loss':[], 'val_loss':[], 'train_acc':[], 'val_acc':[]}
    best_val_acc = 0; best_state = None; patience_counter = 0

    for epoch in range(T):
        model.train(); tl, tc, tt = 0, 0, 0
        for bx, by in loader:
            optimizer.zero_grad()
            pred = model(bx)
            loss = criterion(pred, by); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
            optimizer.step()
            tl += loss.item()*len(bx); tc += ((pred>0.5).float()==by).sum().item(); tt += len(bx)
        model.eval()
        with torch.no_grad():
            vp = model(X_va)
            vl = criterion(vp, y_va).item()
            va = ((vp>0.5).float()==y_va).float().mean().item()
        history['train_loss'].append(tl/tt); history['val_loss'].append(vl)
        history['train_acc'].append(tc/tt);  history['val_acc'].append(va)
        scheduler.step()

        if va > best_val_acc:
            best_val_acc = va; best_state = copy.deepcopy(model.state_dict()); patience_counter = 0
        else:
            patience_counter += 1

        if verbose and (epoch%10==0 or epoch==T-1):
            print(f'  Ep {epoch:>3} | Loss: {tl/tt:.4f} | Val Acc: {100*va:.2f}% | Best: {100*best_val_acc:.2f}%')
        if patience_counter >= config.get('patience', T):
            if verbose: print(f'  Early stopping at epoch {epoch}')
            break

    model.load_state_dict(best_state)
    if verbose: print(f'  Best Val Acc: {100*best_val_acc:.2f}%')
    return model, history

print('train_model() ready — 1D-CNN, patience=40.')

In [ ]:
def evaluate(model, X, y_true):
    """Returns metrics dict for a given CNN model and data."""
    model.eval()
    X_t = torch.tensor(X).to(device)
    with torch.no_grad():
        pred = model(X_t)
    y_prob = pred.cpu().numpy().flatten()
    y_pred = (y_prob > 0.5).astype(float)
    y_true = y_true.flatten()
    rep  = classification_report(y_true, y_pred, target_names=['LOS','NLOS'],
                                 output_dict=True, zero_division=0)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    return {
        'acc':      rep['accuracy'],
        'f1_macro': rep['macro avg']['f1-score'],
        'f1_los':   rep['LOS']['f1-score'],
        'f1_nlos':  rep['NLOS']['f1-score'],
        'auc':      auc(fpr, tpr),
        'fpr': fpr, 'tpr': tpr,
        'cm':  confusion_matrix(y_true, y_pred),
    }

print('evaluate() helper ready (1D-CNN).')

---
## Section 4: Single 70/15/15 Split Comparison
Quick baseline comparison using one fixed split.

In [ ]:
def run_experiment(csv_name, label, config=CONFIG, seed=42):
    print(f"\n{'='*60}\nExperiment: {label}\n{'='*60}")
    X_all, y_all = load_cir_dataset(DATA_DIR + csv_name)
    X_tr, X_tmp, y_tr, y_tmp = train_test_split(
        X_all, y_all, test_size=config['val_ratio']+config['test_ratio'],
        stratify=y_all, random_state=seed)
    X_va, X_te, y_va, y_te = train_test_split(
        X_tmp, y_tmp,
        test_size=config['test_ratio']/(config['val_ratio']+config['test_ratio']),
        stratify=y_tmp, random_state=seed)
    print(f'  Train={len(X_tr)}, Val={len(X_va)}, Test={len(X_te)}')
    model, history = train_model(X_tr, y_tr, X_va, y_va, config=config)
    m = evaluate(model, X_te, y_te)
    m['label'] = label; m['history'] = history
    print(f"\n  Test Acc={m['acc']:.4f} | Macro F1={m['f1_macro']:.4f} | AUC={m['auc']:.4f}")
    return m


results_single = run_experiment('single_channel5_dataset.csv',  'Single-Channel (c5 only)')
results_multi  = run_experiment('multi_channel4_dataset.csv',   'Multi-Channel (c1,c3,c4,c7)')

In [ ]:
# ── Single-split plots ──────────────────────────────────────────────────
_res = [results_single, results_multi]
_colors = ['#3498db', '#e74c3c']
width = 0.3

fig, axs = plt.subplots(1, 4, figsize=(24, 6))

# Bar: key metrics
ax = axs[0]
mk = ['acc','f1_macro','auc']; mn = ['Accuracy','Macro F1','AUC']
for i, (r, c) in enumerate(zip(_res, _colors)):
    vals = [r[k] for k in mk]
    bars = ax.bar(np.arange(3)+i*width, vals, width, label=r['label'], color=c, alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.005,
                f'{v:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')
ax.set_xticks(np.arange(3)+width/2); ax.set_xticklabels(mn)
ax.set_ylim([0.4,1.12]); ax.set_title('Metric Comparison', fontweight='bold')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3, axis='y')

# ROC
ax = axs[1]
for r, c in zip(_res, _colors):
    ax.plot(r['fpr'], r['tpr'], color=c, lw=2, label=f"{r['label']} (AUC={r['auc']:.3f})")
ax.plot([0,1],[0,1],'k--',lw=1,alpha=0.5)
ax.set_title('ROC Curves', fontweight='bold'); ax.set_xlabel('FPR'); ax.set_ylabel('TPR')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

# Val acc curves
ax = axs[2]
for r, c in zip(_res, _colors):
    ax.plot(r['history']['val_acc'], color=c, lw=2, label=r['label'])
ax.set_title('Val Accuracy per Epoch', fontweight='bold')
ax.set_xlabel('Epoch'); ax.set_ylabel('Val Accuracy')
ax.set_ylim([0.4,1.05]); ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

# Per-class F1
ax = axs[3]
for i, (r, c) in enumerate(zip(_res, _colors)):
    vals = [r['f1_los'], r['f1_nlos']]
    bars = ax.bar(np.arange(2)+i*width, vals, width, label=r['label'], color=c, alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.005,
                f'{v:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')
ax.set_xticks(np.arange(2)+width/2); ax.set_xticklabels(['LOS F1','NLOS F1'])
ax.set_ylim([0.4,1.12]); ax.set_title('Per-Class F1', fontweight='bold')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('1D-CNN Baseline — Stage 1: Single-Channel vs Multi-Channel — Single Split',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

print('\n' + '='*55)
print(f"{'Metric':<20} {'Single-Ch':>12} {'Multi-Ch':>12}  Delta")
print('='*55)
for k, n in zip(['acc','f1_los','f1_nlos','f1_macro','auc'],
                ['Accuracy','F1 LOS','F1 NLOS','Macro F1','AUC']):
    sv=results_single[k]; mv=results_multi[k]; d=mv-sv
    print(f"{n:<20} {sv:>12.4f} {mv:>12.4f}   {'▲' if d>0 else '▼'}{abs(d):.4f}")
print('='*55)

---
## Section 5: Stratified 5-Fold Cross-Validation
More reliable estimate: averages over 5 different train/test splits.
Epoch prints are suppressed — only one summary line per fold is shown.

In [ ]:
def run_kfold(csv_name, label, n_splits=5, config=CONFIG, seed=42):
    print(f"\n{'='*60}\n5-Fold CV: {label}\n{'='*60}")
    X_all, y_all = load_cir_dataset(DATA_DIR + csv_name)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold_metrics = []
    for fold, (tv_idx, te_idx) in enumerate(skf.split(X_all, y_all)):
        X_tv, X_te = X_all[tv_idx], X_all[te_idx]
        y_tv, y_te = y_all[tv_idx], y_all[te_idx]
        X_tr, X_va, y_tr, y_va = train_test_split(
            X_tv, y_tv, test_size=0.15, stratify=y_tv, random_state=seed)
        fold_seed = seed + fold
        with contextlib.redirect_stdout(io.StringIO()):
            model, _ = train_model(X_tr, y_tr, X_va, y_va,
                                   config=config, verbose=False, seed=fold_seed)
        fm = evaluate(model, X_te, y_te)
        fold_metrics.append(fm)
        collapsed = " [COLLAPSED]" if fm['acc'] <= 0.51 else ""
        print(f"  Fold {fold+1}/{n_splits} | Acc={fm['acc']:.4f} | F1={fm['f1_macro']:.4f} | AUC={fm['auc']:.4f}{collapsed}")
    summary = {'label': label}
    print(f"\n  {'─'*45}")
    print(f"  {'Metric':<12} {'Mean':>8} {'Std':>8}")
    print(f"  {'─'*45}")
    for key in ['acc','f1_macro','f1_los','f1_nlos','auc']:
        vals = np.array([m[key] for m in fold_metrics])
        summary[key] = {'mean': vals.mean(), 'std': vals.std(), 'all': vals.tolist()}
        print(f"  {key:<12} {vals.mean():>8.4f} {vals.std():>8.4f}")
    print(f"  {'─'*45}")
    return summary


kfold_single = run_kfold('single_channel5_dataset.csv', 'Single-Channel (c5 only)')
kfold_multi  = run_kfold('multi_channel4_dataset.csv',  'Multi-Channel (c1,c3,c4,c7)')

In [ ]:
# ── K-Fold comparison plots ────────────────────────────────────────────
_kf = [kfold_single, kfold_multi]
_colors = ['#3498db', '#e74c3c']
_mk = ['acc','f1_macro','f1_los','f1_nlos','auc']
_mn = ['Accuracy','Macro F1','F1 LOS','F1 NLOS','AUC']
width = 0.3

fig, axs = plt.subplots(1, 2, figsize=(18, 6))

# Bar with error bars
ax = axs[0]
x = np.arange(len(_mk))
for i, (kf, c) in enumerate(zip(_kf, _colors)):
    means = [kf[m]['mean'] for m in _mk]
    stds  = [kf[m]['std']  for m in _mk]
    bars = ax.bar(x+i*width, means, width, yerr=stds, label=kf['label'],
                  color=c, alpha=0.85, capsize=4, error_kw={'elinewidth':1.5})
    for bar, m, s in zip(bars, means, stds):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+s+0.008,
                f'{m:.3f}', ha='center', va='bottom', fontsize=7.5, fontweight='bold')
ax.set_xticks(x+width/2); ax.set_xticklabels(_mn, rotation=10)
ax.set_ylim([0.4,1.15]); ax.set_title('5-Fold CV: Mean ± Std', fontweight='bold')
ax.set_ylabel('Score'); ax.legend(fontsize=9); ax.grid(True, alpha=0.3, axis='y')

# Box plots
ax = axs[1]
all_data, all_colors, positions = [], [], []
for mi, metric in enumerate(_mk):
    for ki, (kf, c) in enumerate(zip(_kf, _colors)):
        positions.append(mi*3 + ki)
        all_data.append(kf[metric]['all'])
        all_colors.append(c)
bp = ax.boxplot(all_data, positions=positions, widths=0.7, patch_artist=True,
                medianprops={'color':'black','linewidth':2})
for patch, c in zip(bp['boxes'], all_colors):
    patch.set_facecolor(c); patch.set_alpha(0.7)
ax.set_xticks([mi*3+0.5 for mi in range(len(_mk))])
ax.set_xticklabels(_mn, rotation=10)
ax.set_title('Score Distribution per Fold (blue=single, red=multi)', fontweight='bold')
ax.set_ylabel('Score'); ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('1D-CNN Baseline — Stage 1: Stratified 5-Fold CV — Single vs Multi Channel',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

print('\n' + '='*65)
print(f"{'Metric':<12} {'Single Mean±Std':>20} {'Multi Mean±Std':>20}  Delta")
print('='*65)
for k, n in zip(_mk, _mn):
    sm,ss = kfold_single[k]['mean'], kfold_single[k]['std']
    mm,ms = kfold_multi[k]['mean'],  kfold_multi[k]['std']
    d = mm - sm
    print(f"{n:<12} {sm:>8.4f}±{ss:.4f}      {mm:>8.4f}±{ms:.4f}   {'▲' if d>0 else '▼'}{abs(d):.4f}")
print('='*65)